<a href="https://colab.research.google.com/github/SanthoshiRavi/Healthcare-Text-Processing-/blob/main/Healthcare1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install nltk

In [ ]:
import requests

def fetch_pubmed_article_ids(query, api_key, max_results=50):
    url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmax={max_results}&api_key={api_key}&retmode=json'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        ids = data['esearchresult']['idlist']
        return ids
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Your API key
api_key = '14b3330cab5a4aabaefc418267dd19492909'
# Query term
query = 'diabetes treatment'
# Fetch article IDs
article_ids = fetch_pubmed_article_ids(query, api_key)
print(article_ids)


['38760852', '38760705', '38760678', '38760661', '38760632', '38760595', '38760456', '38760452', '38760427', '38760383', '38760126', '38760125', '38760053', '38760035', '38760033', '38759989', '38759983', '38759874', '38759763', '38759722', '38759707', '38759658', '38759515', '38759430', '38759418', '38759318', '38759315', '38759226', '38759099', '38758937', '38758936', '38758907', '38758866', '38758863', '38758855', '38758848', '38758847', '38758689', '38758687', '38758679', '38758678', '38758675', '38758674', '38758639', '38758517', '38758435', '38758423', '38758248', '38758212', '38758211']


In [ ]:
def fetch_pubmed_article_details(article_ids, api_key):
    ids = ','.join(article_ids)
    url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={ids}&api_key={api_key}&retmode=xml'
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

article_details_xml = fetch_pubmed_article_details(article_ids, api_key)
print(article_details_xml)  # This will be in XML format


<?xml version="1.0" ?>
<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2024//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_240101.dtd">
<PubmedArticleSet>
<PubmedArticle><MedlineCitation Status="PubMed-not-MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">38760852</PMID><DateRevised><Year>2024</Year><Month>05</Month><Day>17</Day></DateRevised><Article PubModel="Electronic"><Journal><ISSN IssnType="Print">1758-5996</ISSN><JournalIssue CitedMedium="Print"><Volume>16</Volume><Issue>1</Issue><PubDate><Year>2024</Year><Month>May</Month><Day>17</Day></PubDate></JournalIssue><Title>Diabetology &amp; metabolic syndrome</Title><ISOAbbreviation>Diabetol Metab Syndr</ISOAbbreviation></Journal><ArticleTitle>Response to semaglutide of non-drinker subjects with type 2 diabetes.</ArticleTitle><Pagination><StartPage>103</StartPage><MedlinePgn>103</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10.1186/s13098-024-01344-6</ELocationID><Abst

In [ ]:
from bs4 import BeautifulSoup

def parse_article_details(article_xml):
    soup = BeautifulSoup(article_xml, 'xml')
    articles = soup.find_all('PubmedArticle')
    parsed_data = []
    for article in articles:
        title = article.find('ArticleTitle').text if article.find('ArticleTitle') else ""
        abstract = article.find('AbstractText').text if article.find('AbstractText') else ""
        parsed_data.append((title, abstract))
    return parsed_data

parsed_articles = parse_article_details(article_details_xml)
for title, abstract in parsed_articles:
    print(f"Title: {title}\nAbstract: {abstract}\n")


Title: Response to semaglutide of non-drinker subjects with type 2 diabetes.
Abstract: Metabolic dysfunction-associated Steatotic Liver Disease (MASLD) displays a worse prognosis in subjects with type 2 diabetes (T2D); effective treatments are, so far, scanty. Semaglutide showed efficacy in improving steatohepatitis. We longitudinally observed a MASLD cohort of T2D subjects starting semaglutide, to detect an improvement of non-invasive surrogates of steatosis and fibro-inflammatory liver involvement, evaluating the role of mild alcohol consumption.

Title: Characterization of the gut microbiota in polycystic ovary syndrome with dyslipidemia.
Abstract: Polycystic ovary syndrome (PCOS) is an endocrinopathy in childbearing-age females which can cause many complications, such as diabetes, obesity, and dyslipidemia. The metabolic disorders in patients with PCOS were linked to gut microbial dysbiosis. However, the correlation between the gut microbial community and dyslipidemia in PCOS remai

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

preprocessed_articles = [(preprocess_text(title), preprocess_text(abstract)) for title, abstract in parsed_articles]
for title, abstract in preprocessed_articles:
    print(f"Title: {title}\nAbstract: {abstract}\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Title: Response semaglutide nondrinker subject type diabetes
Abstract: Metabolic dysfunctionassociated Steatotic Liver Disease MASLD display worse prognosis subject type diabetes TD effective treatment far scanty Semaglutide showed efficacy improving steatohepatitis longitudinally observed MASLD cohort TD subject starting semaglutide detect improvement noninvasive surrogate steatosis fibroinflammatory liver involvement evaluating role mild alcohol consumption

Title: Characterization gut microbiota polycystic ovary syndrome dyslipidemia
Abstract: Polycystic ovary syndrome PCOS endocrinopathy childbearingage female cause many complication diabetes obesity dyslipidemia metabolic disorder patient PCOS linked gut microbial dysbiosis However correlation gut microbial community dyslipidemia PCOS remains unillustrated study elucidated different gut microbiota patient PCOS dyslipidemia PCOSD compared PCOS healthy woman

Title: AMPK activation eliminates senescent cell diabetic wound inducing N

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine title and abstract for each article
combined_texts = [f"{title} {abstract}" for title, abstract in preprocessed_articles]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the combined texts
X = vectorizer.fit_transform(combined_texts)

# Print the shape of the TF-IDF matrix
print(f"TF-IDF matrix shape: {X.shape}")

# Print feature names (optional, but useful for understanding the feature space)
feature_names = vectorizer.get_feature_names_out()
print(f"Number of features: {len(feature_names)}")
print(f"First 10 features: {feature_names[:10]}")


TF-IDF matrix shape: (50, 1697)
Number of features: 1697
First 10 features: ['abelmoschus' 'aberrant' 'abg' 'ability' 'abnormality' 'absolute'
 'acceleration' 'acceptability' 'acceptable' 'accessible']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Combine title and abstract for each article
combined_texts = [f"{title} {abstract}" for title, abstract in preprocessed_articles]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the combined texts
X = vectorizer.fit_transform(combined_texts)

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Print the shape of the TF-IDF matrix
print(f"TF-IDF matrix shape: {X.shape}")

# Print the entire TF-IDF matrix
print(tfidf_df)


TF-IDF matrix shape: (50, 1697)
    abelmoschus  aberrant       abg   ability  abnormality  absolute  \
0      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
1      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
2      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
3      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
4      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
5      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
6      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
7      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
8      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
9      0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
10     0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
11     0.000000  0.000000  0.000000  0.000000     0.000000  0.000000   
12     0.000000  0.000000  0.000